# Temporal Ordering

When prompts contain events, conversations, or facts with temporal relationships, presenting them in chronological order helps language models better understand causality and event sequences. Models process narratives more effectively when events flow from past to present, making it easier to identify cause-and-effect relationships and understand "what happened when and why."

Temporal ordering is simply organizing time-dependent information chronologically. In practice, this usually means sorting events or messages by their timestamp. This straightforward technique dramatically improves model comprehension of event sequences and causal relationships.

This notebook demonstrates the practical application of temporal ordering in AI agent context engineering, showing the problem, the simple solution, and real-world use cases.

**Common use cases:**
1. **Conversation history**: Order messages chronologically for natural flow.
2. **Event logs**: Sort incidents/alerts by time for root cause analysis.
3. **Document versions**: Present history from oldest to newest.
4. **Multi-turn interactions**: Maintain temporal context across turns.

In [1]:
# Import required libraries
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from datetime import datetime
import os

### Initialize LLM

In [2]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY", "").strip(), temperature=0)

## Demonstrating the problem: chronological vs non-ncronological
Let's empirically show how chronological ordering affects LLM comprehension. We will present the same incident timeline in two ways - shuffled and chronological - and compare the model's ability to identify root cause and event sequence.

In [3]:
# Same events, different orders
shuffled_timeline = """
Incident Timeline:

15:45 - System fully restored
14:20 - Database failover initiated
15:15 - Engineers identified corrupted index
13:45 - Customer complaints about slow page loads
14:00 - Monitoring alerts: High database CPU (95%)
14:45 - Primary database became unresponsive
15:30 - Index rebuild completed
13:30 - Automated backup process started
13:25 - Marketing campaign email sent to 500,000 users
14:30 - Application servers returning 500 errors

Question: What was the root cause and sequence of events?
"""

chronological_timeline = """
Incident Timeline (Chronological):

13:25 - Marketing campaign email sent to 500,000 users
13:30 - Automated backup process started
13:45 - Customer complaints about slow page loads
14:00 - Monitoring alerts: High database CPU (95%)
14:20 - Database failover initiated
14:30 - Application servers returning 500 errors
14:45 - Primary database became unresponsive
15:15 - Engineers identified corrupted index
15:30 - Index rebuild completed
15:45 - System fully restored

Question: What was the root cause and sequence of events?
"""

# Test both versions
print("SHUFFLED ORDER - Model must reconstruct timeline:")
print("=" * 60)
response_shuffled = llm.invoke([HumanMessage(content=shuffled_timeline)])
print(response_shuffled.content[:400], "...\n")

print("\nCHRONOLOGICAL ORDER - Clear causal chain:")
print("=" * 60)
response_chrono = llm.invoke([HumanMessage(content=chronological_timeline)])
print(response_chrono.content[:400], "...\n")

SHUFFLED ORDER - Model must reconstruct timeline:
Based on the incident timeline provided, the root cause and sequence of events can be summarized as follows:

### Root Cause:
The root cause of the incident appears to be a corrupted database index, which led to high CPU usage and ultimately caused the primary database to become unresponsive. This corruption likely resulted from the load generated by the marketing campaign email sent to 500,000 us ...


CHRONOLOGICAL ORDER - Clear causal chain:
The root cause of the incident appears to be related to the high database CPU usage, which likely led to the database becoming unresponsive. Here’s the sequence of events that contributed to the incident:

1. **13:25** - A marketing campaign email was sent to 500,000 users. This likely increased traffic to the application as users began to engage with the content.
   
2. **13:30** - An automated b ...



Impact:
- Chronological ordering improves root cause identification.
- Reduces event sequence errors.
- Makes causal relationships immediately apparent.

## The solution: timestamp sorting
In production AI agent systems, temporal ordering is almost always just sorting by timestamp. Just organize our context chronologically before sending to the LLM.

In [4]:
# This is what temporal ordering looks like in practice
from typing import List, Dict

def order_events_chronologically(events: List[Dict]) -> List[Dict]:
    """
    Sort events by timestamp. This is 95% of temporal ordering in production.
    
    Args:
        events: List of event dictionaries with 'timestamp' and 'description' keys
        
    Returns:
        Events sorted chronologically (oldest to newest)
    """
    return sorted(events, key=lambda e: e['timestamp'])


# Example usage with real events
events = [
    {"timestamp": datetime(2024, 1, 15, 15, 45), "description": "System restored"},
    {"timestamp": datetime(2024, 1, 15, 14, 20), "description": "Failover initiated"},
    {"timestamp": datetime(2024, 1, 15, 13, 25), "description": "Marketing email sent"},
    {"timestamp": datetime(2024, 1, 15, 14, 00), "description": "High CPU alert"},
]

# Sort chronologically
sorted_events = order_events_chronologically(events)

print("Events in chronological order:")
print("=" * 60)
for event in sorted_events:
    time_str = event['timestamp'].strftime("%H:%M")
    print(f"{time_str} - {event['description']}")

Events in chronological order:
13:25 - Marketing email sent
14:00 - High CPU alert
14:20 - Failover initiated
15:45 - System restored


## Real-world use case: conversation history in AI agents
The most common application of temporal ordering in AI agents is organizing conversation history. When building an agent with memory, we want to present messages in chronological order so the LLM can follow the conversation flow naturally.

In [5]:
# Simulating a conversation history from a database or memory store
# In practice, these might come from different retrieval queries and be out of order
conversation_history = [
    {"timestamp": datetime(2024, 1, 15, 10, 30), "role": "user", "content": "What's the status of the deployment?"},
    {"timestamp": datetime(2024, 1, 15, 10, 32), "role": "assistant", "content": "The deployment completed at 10:15 AM"},
    {"timestamp": datetime(2024, 1, 15, 10, 15), "role": "user", "content": "Can you start the deployment?"},
    {"timestamp": datetime(2024, 1, 15, 10, 16), "role": "assistant", "content": "Deployment started. I'll monitor progress."},
    {"timestamp": datetime(2024, 1, 15, 10, 35), "role": "user", "content": "Were there any errors?"},
]

# Sort chronologically (oldest to newest is standard for conversation history)
ordered_conversation = sorted(conversation_history, key=lambda msg: msg['timestamp'])

# Build context for LLM
context = "Previous conversation:\n\n"
for msg in ordered_conversation:
    time_str = msg['timestamp'].strftime("%H:%M")
    context += f"[{time_str}] {msg['role']}: {msg['content']}\n"

print("Chronologically ordered conversation context:")
print("=" * 60)
print(context)

Chronologically ordered conversation context:
Previous conversation:

[10:15] user: Can you start the deployment?
[10:16] assistant: Deployment started. I'll monitor progress.
[10:30] user: What's the status of the deployment?
[10:32] assistant: The deployment completed at 10:15 AM
[10:35] user: Were there any errors?



Why this matters:
- LLM can follow the natural conversation flow.
- User's questions make sense in context ("Were there any errors?" comes after deployment).
- Agent can provide consistent, contextually appropriate responses.

## Handling edge cases: missing timestamps
Sometimes events don't have explicit timestamps. Here is a practical approach for handling this common scenario.

In [6]:
from typing import Optional

def order_events_with_fallback(events: List[Dict]) -> List[Dict]:
    """
    Sort events by timestamp, using insertion order for events without timestamps.
    This handles the common case where some events lack explicit time markers.
    
    Args:
        events: List of event dicts with optional 'timestamp' key
        
    Returns:
        Events sorted chronologically where possible
    """
    # Separate events with and without timestamps
    timestamped = [e for e in events if 'timestamp' in e and e['timestamp'] is not None]
    no_timestamp = [e for e in events if 'timestamp' not in e or e['timestamp'] is None]
    
    # Sort timestamped events
    timestamped.sort(key=lambda e: e['timestamp'])
    
    # Append non-timestamped events at the end (maintain insertion order)
    return timestamped + no_timestamp


# Example with mixed timestamp availability
mixed_events = [
    {"timestamp": datetime(2024, 1, 15, 14, 30), "description": "Error detected"},
    {"description": "User reported slow performance"},  # No timestamp
    {"timestamp": datetime(2024, 1, 15, 14, 15), "description": "Load spike began"},
    {"description": "Team investigating"},  # No timestamp
]

ordered = order_events_with_fallback(mixed_events)

print("Events ordered with fallback handling:")
print("=" * 60)
for event in ordered:
    if 'timestamp' in event and event['timestamp']:
        time_str = event['timestamp'].strftime("%H:%M")
        print(f"{time_str} - {event['description']}")
    else:
        print(f"[No timestamp] - {event['description']}")

Events ordered with fallback handling:
14:15 - Load spike began
14:30 - Error detected
[No timestamp] - User reported slow performance
[No timestamp] - Team investigating


Strategy:
- Events with timestamps come first, in chronological order.
- Events without timestamps maintain insertion order.
- Simple, predictable behavior for edge cases.

## Choosing order direction: oldest-first vs newest-first
Depending on our use case, we might want oldest-first (default) or newest-first (recency bias).

In [7]:
def order_events(events: List[Dict], newest_first: bool = False) -> List[Dict]:
    """
    Sort events chronologically with configurable direction.
    
    Args:
        events: List of event dictionaries with 'timestamp' key
        newest_first: If True, sort newest to oldest. If False, oldest to newest.
        
    Returns:
        Chronologically sorted events
    """
    return sorted(events, key=lambda e: e['timestamp'], reverse=newest_first)


# Same events, different ordering strategies
events = [
    {"timestamp": datetime(2024, 1, 15, 10, 00), "description": "System started"},
    {"timestamp": datetime(2024, 1, 15, 10, 30), "description": "First user login"},
    {"timestamp": datetime(2024, 1, 15, 11, 00), "description": "Peak traffic reached"},
]

print("OLDEST-FIRST (default for narratives):")
print("=" * 60)
for event in order_events(events, newest_first=False):
    print(f"{event['timestamp'].strftime('%H:%M')} - {event['description']}")

print("\nNEWEST-FIRST (for recency bias):")
print("=" * 60)
for event in order_events(events, newest_first=True):
    print(f"{event['timestamp'].strftime('%H:%M')} - {event['description']}")

OLDEST-FIRST (default for narratives):
10:00 - System started
10:30 - First user login
11:00 - Peak traffic reached

NEWEST-FIRST (for recency bias):
11:00 - Peak traffic reached
10:30 - First user login
10:00 - System started


When to use each:
- Oldest-first: Conversations, incident timelines, narratives.
- Newest-first: Recent activity summaries, recency-weighted context.